In [83]:
import geopandas as gpd
import laspy
import pdal
import json,os
import pandas as pd

In [94]:
def load_geolidar(boundary,area,filename):
    pipeline = {
        "pipeline": [
            {
    "bounds": f"{boundary}",
    "filename": f"https://s3-us-west-2.amazonaws.com/usgs-lidar-public/{area}/ept.json",
    "type": "readers.ept",
    "tag": "readdata"
            },
            {
                "limits": "Classification![7:7]",
                "type": "filters.range",
                "tag": "nonoise"
            },
            {
                "assignment": "Classification[:]=0",
                "tag": "wipeclasses",
                "type": "filters.assign"
            },
            {
                "out_srs": "EPSG:26915",
                "tag": "reprojectUTM",
                "type": "filters.reprojection"
            },
            {
                "tag": "groundify",
                "type": "filters.smrf"
            },
            {
                "limits": "Classification[2:2]",
                "type": "filters.range",
                "tag": "classify"
            },
            {
                "filename": f"{filename}.laz",
                "inputs": [ "classify" ],
                "tag": "writerslas",
                "type": "writers.las"
            },
            {
                "filename": f"{filename}.tif",
                "gdalopts": "tiled=yes,     compress=deflate",
                "inputs": [ "writerslas" ],
                "nodata": -9999,
                "output_type": "idw",
                "resolution": 1,
                "type": "writers.gdal",
                "window_size": 6
            }
        ]
    }
    json_object = json.dumps(pipeline, indent = 4)

    # Writing to sample.json
    with open(f"{filename}.json", "w") as outfile:
        outfile.write(json_object)
    
    os.system(f'pdal pipeline {filename}.json --debug')
    pipeline=pdal.Reader(f'{filename}.laz').pipeline()
    count = pipeline.execute()
    arrays = pipeline.arrays
    captured_array = arrays.pop()
    results = [array[['X','Y','Z']][i] for i,x in enumerate(captured_array)]
    df = pd.DataFrame({'elevation_m': [x[2] for x in results]})
    gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy([x[1] for x in results], 
                                                           [x[0] for x in results]))
    return gdf

In [95]:
result = load_geolidar(([-10425171.940, -10423171.940], [5164494.710, 5166494.710]),"IA_FullState","../data/iowa")

(PDAL Debug) Debugging...
(pdal pipeline readers.ept Debug) Query bounds: ([-10425171.94, -10423171.94], [5164494.71, 5166494.71], [-1.797693134862316e+308, 1.797693134862316e+308])
Threads: 15
(pdal pipeline Debug) Executing pipeline in standard mode.
(pdal pipeline filters.smrf Debug) progressiveFilter: radius = 1	2234705 ground cells	3302 non-ground cells	(0.15% of cells contain ground)
(pdal pipeline filters.smrf Debug) progressiveFilter: radius = 1	2026879 ground cells	211128 non-ground cells	(9.43% of cells contain ground)
(pdal pipeline filters.smrf Debug) progressiveFilter: radius = 2	1979134 ground cells	258873 non-ground cells	(11.57% of cells contain ground)
(pdal pipeline filters.smrf Debug) progressiveFilter: radius = 3	1959708 ground cells	278299 non-ground cells	(12.44% of cells contain ground)
(pdal pipeline filters.smrf Debug) progressiveFilter: radius = 4	1948026 ground cells	289981 non-ground cells	(12.96% of cells contain ground)
(pdal pipeline filters.smrf Debug) p

In [96]:
result

,elevation_m,geometry
0,275.27,POINT (4654049.400 447608.700)
1,275.40,POINT (4654049.340 447604.530)
2,275.21,POINT (4654050.430 447609.950)
3,275.18,POINT (4654051.180 447609.190)
4,275.22,POINT (4654050.830 447607.410)
...,...,...
1399938,279.10,POINT (4654064.880 446154.200)
1399939,277.04,POINT (4654064.830 446147.740)
1399940,274.25,POINT (4654064.760 446139.880)
1399941,277.36,POINT (4654064.720 446131.440)
